In [ ]:
from torchvision.transforms import transforms

In [ ]:
import cv2

image = cv2.imread("/content/5a1cf8f3ee8a8.png")
image.shape

(278, 278, 3)

In [ ]:
from torchvision.io import read_image

image = read_image("/content/5a1cf8f3ee8a8.png")
image.shape

torch.Size([4, 278, 278])

In [ ]:
import torch

a = torch.rand(5)

In [ ]:
f_t = transforms.Lambda(lambda x: x * 2)
f_t(a)
transforms.ToPILImage()

tensor([1.8895, 1.8718, 1.8669, 1.9350, 1.2967])

In [ ]:
f_t1 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((124, 124)),
    transforms.PILToTensor()
])
f_t1(a)
transforms.ToPILImage()

tensor([[3.6096, 2.9783, 0.5833, 1.3815, 2.8053]])

In [ ]:
import pathlib
from torch.utils.data import Dataset

In [ ]:
class LatinDataset(Dataset):
  def __init__(self, path_dataset: pathlib.Path):
    self.path_dataset = path_dataset
    self.data_list = [x for x in self.path_dataset.glob("**/*") if x.is_file()]
    self.data_class = list(set(x.parent for x in self.data_list))
    self.transform_func = transforms.Compose([
      transforms.ToPILImage(),
      transforms.Grayscale(),
      transforms.Resize((124, 124)),
      transforms.PILToTensor()
  ])

  def __len__(self):
    return len(self.data_list)

  def __getitem__(self, index):
    img_path = self.data_list[index]
    img_label = torch.zeros(len(self.data_class))
    img_label[self.data_class.index(img_path.parent)] = 1.0
    img_tensor = read_image(img_path)
    return self.transform_func(img_tensor), img_label


In [ ]:
import torch.nn as nn

In [ ]:
import torch

a = torch.rand((2, 1, 1, 28))
a.size()

torch.Size([2, 1, 1, 28])

In [ ]:
a.view(-1, 28).size()

torch.Size([2, 28])

In [ ]:
class CVModel(nn.Module):
  def __init__(self):
    self.conv_1 = nn.Conv2d(1, 32, (3, 3))
    self.conv_2 = nn.Conv2d(32, 64, (3, 3))
    self.pool_1 = nn.MaxPool2d((2, 2))
    self.conv_3 = nn.Conv2d(64, 128, (3, 3))
    self.conv_4 = nn.Conv2d(128, 128, (3, 3))
    self.pool_2 = nn.MaxPool2d((2, 2))
    self.glob_pool = nn.MaxPool2d((28, 28))
    self.linear = nn.Linear(128, 26)
    self.softmax = nn.Softmax()

  def forward(self, X):
    res = self.conv_1(X)
    res = self.conv_2(res)
    res = self.pool_1(res)
    res = self.conv_3(res)
    res = self.conv_4(res)
    res = self.pool_2(res)
    res = self.glob_pool(res)
    res = self.linear(res.view(-1, 128))
    return self.softmax(res)
